###

import PyPDF2

PDFfilename = "./data/190218.pdf" #filename of your PDF/directory where your PDF is stored

pfr = PyPDF2.PdfFileReader(open(PDFfilename, "rb")) #PdfFileReader object

pg0 = pfr.getPage(0) #extract pg 127

writer = PyPDF2.PdfFileWriter() #create PdfFileWriter object
#add pages
writer.addPage(pg0)

NewPDFfilename = "allTables.csv" #filename of your PDF/directory where you want your new PDF to be
with open(NewPDFfilename, "wb") as outputStream:
    writer.write(outputStream) #write pages to new PDF

###

pip install git+https://github.com/pdftables/python-pdftables-api.git

###

pip install https://github.com/pdftables/python-pdftables-api/archive/master.tar.gz
import pdftables_api

c = pdftables_api.Client('zs2k6h51h1t5')
c.xlsx('./data/190220.pdf', '190220plan') #replace c.xlsx with c.csv to convert to CSV

In [2]:
import pandas as pd
import numpy as np

In [3]:
seewith14 = pd.read_csv('190214plan.csv', encoding='euc-kr')
seewith14.head(2)

,집회 날짜,집회 일시,집회 장소,신고 인원,관할서,비고
0,2019.02.14,05:30∼10:00,GS건설현장 신대방동,500,동작,NaN
1,2019.02.14,10:00∼종료시,마포역,130,마포,NaN


###

import csv
import glob
import os
import sys

input_path = "C:\\Users\\student\\Desktop\\seewith"
output_file = 'result.csv' 
first_file = True

for input_file in glob.glob(os.path.join(input_path,'*plan.csv')):
    print(os.path.basename(input_file))
    with open(input_file, 'r', newline='') as csv_in_file:
      with open(output_file, 'a', newline='') as csv_out_file:
        filereader = csv.reader(csv_in_file)
        filewriter = csv.writer(csv_out_file)
        if first_file:
          for row in filereader:
            filewriter.writerow(row)
          first_file = False
        else:
          header = next(filereader)
          for row in filereader:
            filewriter.writerow(row)

In [5]:
seewith = pd.read_csv('190214plan.csv', encoding='euc-kr')
seewith.head(2)
# revise1 = seewith.sort_index(ascending=False)
# revise1.head(2)

,집회 날짜,집회 일시,집회 장소,신고 인원,관할서,비고
0,2019.02.14,05:30∼10:00,GS건설현장 신대방동,500,동작,NaN
1,2019.02.14,10:00∼종료시,마포역,130,마포,NaN


In [6]:
# KAKAO_REST_APIKEY = '14a8c6f41a5695f5cbdf4dc98a50659f'

In [7]:
seewith.columns

Index(['집회 날짜', '집회 일시', '집회 장소', '신고 인원', '관할서', '비고'], dtype='object')

In [8]:
tmp = seewith.pivot_table( index=['관할서'], columns=['집회 날짜'] ) 
tmp
revise=tmp

In [9]:
tmp[:] = tmp[:].fillna(0)
tmp.head()

,신고 인원
집회 날짜,2019.02.14
관할서,
금천,200.0
남대문,50.0
동대문,60.0
동작,500.0
마포,130.0


In [10]:
tmp.index

Index(['금천', '남대문', '동대문', '동작', '마포', '영등포', '용산', '종로'], dtype='object', name='관할서')

In [11]:
tmp.columns

MultiIndex(levels=[['비고', '신고 인원'], ['2019.02.14']],
           labels=[[1], [0]],
           names=[None, '집회 날짜'])

In [12]:
tmp.columns.levels[0]
tmp.columns.levels[1]

Index(['2019.02.14'], dtype='object', name='집회 날짜')

In [13]:
KAKAO_REST_APIKEY = '14a8c6f41a5695f5cbdf4dc98a50659f'

In [14]:
tmp_names = []
# tmp.index.unique()
for n in tmp.index:
    tmp_names.append('서울' + n + '경찰서' )
tmp_names

tmp_names = ['서울' + n + '경찰서' for n in tmp.index]
tmp_names

['서울금천경찰서',
 '서울남대문경찰서',
 '서울동대문경찰서',
 '서울동작경찰서',
 '서울마포경찰서',
 '서울영등포경찰서',
 '서울용산경찰서',
 '서울종로경찰서']

In [15]:
tmp.head(2)

,신고 인원
집회 날짜,2019.02.14
관할서,
금천,200.0
남대문,50.0


In [16]:
# https://dapi.kakao.com/v2/local/search/keyword.json?query=서울중부경찰서
from tqdm import tqdm # 진행율
# 통신(http)을 하기 위한 모듈
from urllib.request import urlopen, Request
import urllib
import json

URL = 'https://dapi.kakao.com/v2/local/search/keyword.json?query='
for tmp in tqdm(tmp_names):
    # print(URL + urllib.parse.quote(tmp))
    # 전문구성 (공백이나 한글을 URL 인코딩 처리를 해야한다)
    req = Request(URL + urllib.parse.quote(tmp))
    req.add_header('Authorization', 'KakaoAK 14a8c6f41a5695f5cbdf4dc98a50659f')
    # 요청
    res = urlopen(req)
    print(res)
    # 응답 데이터 json 파싱 => 200
    if res.code == 200:
        # 파싱
        dummy = json.load(res)
        print('정상응답', dummy['documents'][0]['address_name'],dummy['documents'][0]['x'],dummy['documents'][0]['x'])
        #documents의 첫번째 데이터의 address_name, x, y 출력
        # 파싱 결과로 주소, x, y를 적제
        
    break

  0%|          | 0/8 [00:00<?, ?it/s]

정상응답 서울 금천구 시흥동 산 189 126.89674921449215 126.89674921449215


In [17]:
address = []   # 주소
lats    = []   # 위도
lngs    = []   # 경도

for tmp in tqdm(tmp_names):
    req = Request(URL + urllib.parse.quote(tmp))
    req.add_header('Authorization', 'KakaoAK 14a8c6f41a5695f5cbdf4dc98a50659f')
    res = urlopen(req)
    if res.code == 200:
        dummy = json.load(res)
        # 데이터 적제
        address.append(dummy['documents'][0]['address_name'])
        lats.append(dummy['documents'][0]['x'])
        lngs.append(dummy['documents'][0]['y'])

100%|██████████| 8/8 [00:00<00:00, 10.51it/s]


In [18]:
len(address), len(lats), len(lngs)

(8, 8, 8)

In [19]:
revise['주소'] = address
revise['위도'] = lats
revise['경도'] = lngs

In [20]:
revise2 = revise.loc[:, ['신고 인원','위도','경도'] ]
revise2.head(2)

,신고 인원,위도,경도
집회 날짜,2019.02.14,,
관할서,,,
금천,200.0,126.89674921449215,37.45725314643252
남대문,50.0,126.973499492486,37.5549786547527


In [21]:
revise2.columns.levels[0]

Index(['비고', '신고 인원', '주소', '위도', '경도'], dtype='object')

In [36]:
new_revise2 = [ "%s" % ( col ) 
              for col in revise2.columns.levels[1]
            ]
new_revise3 = ['2019.02.14', '위도', '경도']

In [37]:
revise2.columns = new_revise3
revise2.head()

,2019.02.14,위도,경도
관할서,,,
금천,200.0,126.89674921449215,37.45725314643252
남대문,50.0,126.973499492486,37.5549786547527
동대문,60.0,127.04576239696928,37.58508176321097
동작,500.0,126.94285899341229,37.513147550556226
마포,130.0,126.95394220726662,37.55101229580099


In [39]:
revise2['시위 인원수'] = revise2['2019.02.14']
revise2.head()

,2019.02.14,위도,경도,시위 인원수
관할서,,,,
금천,200.0,126.89674921449215,37.45725314643252,200.0
남대문,50.0,126.973499492486,37.5549786547527,50.0
동대문,60.0,127.04576239696928,37.58508176321097,60.0
동작,500.0,126.94285899341229,37.513147550556226,500.0
마포,130.0,126.95394220726662,37.55101229580099,130.0


In [40]:
revise3 = revise2
del revise3['2019.02.14']

In [41]:
revise3.head(2)

,위도,경도,시위 인원수
관할서,,,
금천,126.89674921449215,37.45725314643252,200.0
남대문,126.973499492486,37.5549786547527,50.0


In [42]:
# address를 이용하면 자치구 => 자치구만 모아서 gus 리스트에 모아라
gus = [ addr.split()[1] for addr in address]
gus

['금천구', '중구', '동대문구', '동작구', '마포구', '영등포구', '용산구', '종로구']

In [43]:
revise3['구별'] = gus
revise3

,위도,경도,시위 인원수,구별
관할서,,,,
금천,126.89674921449215,37.45725314643252,200.000000,금천구
남대문,126.973499492486,37.5549786547527,50.000000,중구
동대문,127.04576239696928,37.58508176321097,60.000000,동대문구
동작,126.94285899341229,37.513147550556226,500.000000,동작구
마포,126.95394220726662,37.55101229580099,130.000000,마포구
영등포,126.90094655076123,37.52585217071908,300.000000,영등포구
용산,126.967778896191,37.5411811395942,500.000000,용산구
종로,126.98489890358965,37.57554203072626,383.333333,종로구


In [44]:
# 주소 추가
revise3['주소'] = address
revise3['위도'] = lats
revise3['경도'] = lngs
revise3.head(2)

,위도,경도,시위 인원수,구별,주소
관할서,,,,,
금천,126.89674921449215,37.45725314643252,200.0,금천구,서울 금천구 시흥동 산 189
남대문,126.973499492486,37.5549786547527,50.0,중구,서울 중구 남대문로5가 561


In [45]:
revise3.index

Index(['금천', '남대문', '동대문', '동작', '마포', '영등포', '용산', '종로'], dtype='object', name='관할서')

In [46]:
revise4 = pd.pivot_table(revise3, index='구별', aggfunc=np.sum)
revise4.head(2)

,시위 인원수
구별,
금천구,200.0
동대문구,60.0


In [47]:
revise4['주간 평균 시위 인원수'] = revise4['시위 인원수']/7
revise4['주소'] = revise3['주소']
revise4['위도'] = revise3['위도']
revise4['경도'] = revise3['경도']
revise4.head()

,시위 인원수,주간 평균 시위 인원수,주소,위도,경도
구별,,,,,
금천구,200.0,28.571429,NaN,NaN,NaN
동대문구,60.0,8.571429,NaN,NaN,NaN
동작구,500.0,71.428571,NaN,NaN,NaN
마포구,130.0,18.571429,NaN,NaN,NaN
영등포구,300.0,42.857143,NaN,NaN,NaN


In [48]:
import json
geo_path = './data/skorea_municipalities_geo_simple.json'
geo_str = json.load(open(geo_path, encoding='utf-8'))
geo_str

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'id': '강동구',
   'properties': {'code': '11250',
    'name': '강동구',
    'name_eng': 'Gangdong-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.11519584981606, 37.557533180704915],
      [127.16683184366129, 37.57672487388627],
      [127.18408792330152, 37.55814280369575],
      [127.16530984307447, 37.54221851258693],
      [127.14672806823502, 37.51415680680291],
      [127.12123165719615, 37.52528270089],
      [127.1116764203608, 37.540669955324965],
      [127.11519584981606, 37.557533180704915]]]}},
  {'type': 'Feature',
   'id': '송파구',
   'properties': {'code': '11240',
    'name': '송파구',
    'name_eng': 'Songpa-gu',
    'base_year': '2013'},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[127.0690698130372, 37.522279423505026],
      [127.10087519791962, 37.524841220167055],
      [127.1116764203608, 37.540669955324965],
      [127.12123165719615, 37.52528270089

In [49]:
import folium

In [50]:
revise3.head()

,위도,경도,시위 인원수,구별,주소
관할서,,,,,
금천,126.89674921449215,37.45725314643252,200.0,금천구,서울 금천구 시흥동 산 189
남대문,126.973499492486,37.5549786547527,50.0,중구,서울 중구 남대문로5가 561
동대문,127.04576239696928,37.58508176321097,60.0,동대문구,서울 동대문구 청량리동 229
동작,126.94285899341229,37.513147550556226,500.0,동작구,서울 동작구 노량진동 72-35
마포,126.95394220726662,37.55101229580099,130.0,마포구,서울 마포구 아현동 618-1


In [51]:
map_seoul = folium.Map(location=[37.571974, 126.9867302], zoom_start=10) 
# 행정구역 그리기
map_seoul.choropleth(geo_data = geo_str, # 행정구역
                     data = revise4['주간 평균 시위 인원수'],    
                     columns = [revise4.index, revise4['주간 평균 시위 인원수']],
                     key_on = 'feature.id', # 행정구역 지도정보 id와 데이터의 state가 일치
                     fill_color = 'PuRd',
                     legend_name = '서울시 시위 규모')
map_seoul

C:\ProgramData\Anaconda3\lib\site-packages\folium\folium.py:432: FutureWarning: The choropleth  method has been deprecated. Instead use the new Choropleth class, which has the same arguments. See the example notebook 'GeoJSON_and_choropleth' for how to do this.
  FutureWarning


In [52]:
# 시각화 패키지
import matplotlib.pyplot as plt
import seaborn as sns

# 주피터에 빌트인
%matplotlib inline

# 한글화
from Common import visualKoSetting as vks
vks()

Malgun Gothic


In [53]:
# 저장
revise3.to_csv('./data/revise.csv', sep=',', encoding='euc-kr')

In [54]:
revise3 = pd.read_csv('./data/revise.csv', encoding='euc-kr', index_col=0)
revise3.head(2)

,위도,경도,시위 인원수,구별,주소
관할서,,,,,
금천,126.896749,37.457253,200.0,금천구,서울 금천구 시흥동 산 189
남대문,126.973499,37.554979,50.0,중구,서울 중구 남대문로5가 561


In [56]:
map_seoul = folium.Map(location=[37.571974, 126.9867302], zoom_start=10) 
# 행정구역 그리기
map_seoul.choropleth(geo_data = geo_str, # 행정구역
                     data = revise4['주간 평균 시위 인원수'],    
                     columns = [revise4.index, revise4['주간 평균 시위 인원수']],
                     key_on = 'feature.id', # 행정구역 지도정보 id와 데이터의 state가 일치
                     fill_color = 'PuRd',
                     legend_name = '서울시 시위 규모')

for n, v in enumerate(revise3.index):
    folium.CircleMarker( [ revise3['경도'][n],
                           revise3['위도'][n] ], 
                           radius=revise3['시위 인원수'][n]*0.05,
                           color='#124578',
                           fill_color='#ff0000',
                           fill=True ).add_to(map_seoul)

# 관할서 마킹
for n,v in enumerate(revise3.index):
    #print( revise3['위도'][n], revise3['경도'][n] )
    folium.Marker( [ revise3['경도'][n],
                     revise3['위도'][n]
                    ], popup="서울"+v+'경찰서' ).add_to(map_seoul)
    

    
map_seoul
